In [3]:
import gensim
import logging
logging.root.handlers = []  # Jupyter messes up logging so needs a reset
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
from smart_open import smart_open
import pandas as pd
import numpy as np
from numpy import random
import gensim
import nltk
from sklearn.cross_validation import train_test_split
from sklearn import linear_model
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
from gensim.models import Word2Vec
from sklearn.neighbors import KNeighborsClassifier
from sklearn import linear_model
from nltk.corpus import stopwords
import os, codecs
%matplotlib inline
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords
from sklearn import metrics as skmetrics
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier

/usr/lib64/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [4]:
model = gensim.models.Word2Vec.load("/home/bahbbc/workspace/masters-big5/wiki.pt-br.word2vec.model")
model.init_sims(replace=True)

2017-08-24 23:52:45,306 : INFO : loading Word2Vec object from /home/bahbbc/workspace/masters-big5/wiki.pt-br.word2vec.model
2017-08-24 23:52:47,385 : INFO : loading wv recursively from /home/bahbbc/workspace/masters-big5/wiki.pt-br.word2vec.model.wv.* with mmap=None
2017-08-24 23:52:47,386 : INFO : loading syn0 from /home/bahbbc/workspace/masters-big5/wiki.pt-br.word2vec.model.wv.syn0.npy with mmap=None
2017-08-24 23:52:49,230 : INFO : loading syn1neg from /home/bahbbc/workspace/masters-big5/wiki.pt-br.word2vec.model.syn1neg.npy with mmap=None
2017-08-24 23:52:51,079 : INFO : setting ignored attribute syn0norm to None
2017-08-24 23:52:51,080 : INFO : setting ignored attribute cum_table to None
2017-08-24 23:52:51,082 : INFO : loaded /home/bahbbc/workspace/masters-big5/wiki.pt-br.word2vec.model
2017-08-24 23:52:53,036 : INFO : precomputing L2-norms of word weight vectors


In [5]:
num_features = 400

### Verify model with personality

In [6]:
df = pd.read_csv('~/personality-normalized-single-file-word2vec.csv', encoding='utf-8')
df.shape

/usr/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(238970, 186)

In [7]:
df.extraversion_m.value_counts()

0    121670
1    117300
Name: extraversion_m, dtype: int64

### Word2Vec model training

In [8]:
def word_averaging(wv, words):
    all_words, mean = set(), []
    #print words.shape
    for word in words:
        if isinstance(word, np.ndarray):
            mean.append(word)
        elif word in wv.wv.vocab:
            mean.append(wv.wv.syn0norm[wv.wv.vocab[word].index])
            all_words.add(wv.wv.vocab[word].index)
        #print mean

    if not mean:
        logging.warning("cannot compute similarity with no input %s", words)
        # FIXME: remove these examples in pre-processing
        return np.zeros(num_features,)

    mean = gensim.matutils.unitvec(np.array(mean).mean(axis=0)).astype(np.float32)
    return mean

def  word_averaging_list(wv, text_list):
    return np.vstack([word_averaging(wv, review) for review in text_list ])

In [9]:
def w2v_tokenize_text(text):
    tokens = []
    if text is np.nan:
        return []
    for sent in nltk.sent_tokenize(text, language='portuguese'):
        for word in nltk.word_tokenize(sent, language='portuguese'):
            if len(word) < 2:
                continue
            tokens.append(word)
    return tokens

In [10]:
data = df.apply(lambda r: w2v_tokenize_text(r['formatted_text']), axis=1).values

In [ ]:
%%time
data_word_average = word_averaging_list(model, data)

In [ ]:
clf = RandomForestClassifier(max_depth=2, random_state=42, criterion= "gini", class_weight="balanced")

## Extraversion

In [ ]:
scores = cross_val_score(clf, data_word_average, df['extraversion_m'], cv=10, scoring='f1_macro')

In [ ]:
print("F1-score: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

## Agreableness

In [ ]:
scores = cross_val_score(clf, data_word_average, df['agreeabeness_m'], cv=10, scoring='f1_macro')

In [ ]:
print("F1-score: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

## Consciousness

In [ ]:
scores = cross_val_score(clf, data_word_average, df['conscientiousness_m'], cv=10, scoring='f1_macro')

In [ ]:
print("F1-score: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

## Neuroticism

In [ ]:
scores = cross_val_score(clf, data_word_average, df['neuroticism_m'], cv=10, scoring='f1_macro')

In [ ]:
print("F1-score: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

## Openess

In [ ]:
scores = cross_val_score(clf, data_word_average, df['openness_m'], cv=10, scoring='f1_macro')

In [ ]:
print("F1-score: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))